In [ ]:
!pip3 install -U python-dotenv

In [ ]:
from dotenv import load_dotenv
load_dotenv()

In [ ]:
import datetime
import os
import requests

import pandas as pd
from sqlalchemy import create_engine

## Connect to DB

In [ ]:
engine = create_engine('sqlite:///db.sqlite', echo=False)

## Get Existing Data

In [ ]:
consumedCaloriesDB = pd.read_sql_table(
    'ConsumedCalories',
    con=engine,
    index_col='Date',
    coerce_float=True,
    columns=[
        'Calories',
        'Fat (g)',
        'Carbohydrates (g)',
        'Protein (g)',
    ],
    parse_dates=[
        'Date',
    ]
) if engine.has_table('ConsumedCalories') else pd.DataFrame(columns=['Date', 'Calories', 'Fat (g)', 'Carbohydrates (g)', 'Protein (g)']).set_index('Date')

In [ ]:
exerciseSummaryDB = pd.read_sql_table(
    'ExerciseSummary',
    con=engine,
    index_col='Date',
    coerce_float=True,
    columns=[
        'Exercise Calories',
        'Exercise Minutes'
    ],
    parse_dates=[
        'Date',
    ]
) if engine.has_table('ExerciseSummary') else pd.DataFrame(columns=['Date', 'Exercise Calories', 'Exercise Minutes']).set_index('Date')

## Merge with data from API

In [ ]:
headers = {'cookie': os.getenv('MYFITNESSPAL_COOKIE')}

def get_data(category, report_name, column_name=None):
    data = pd.DataFrame(
        requests.get(
            f'https://www.myfitnesspal.com/reports/results/{category}/{report_name}/7.json',
            headers=headers,
            params={'report_name': report_name},
        ).json()['data']
    )
    data.columns = ['Date', report_name if column_name is None else column_name]
    return data

In [ ]:
calories = get_data(category='nutrition', report_name='Calories', column_name='Calories')

In [ ]:
carbs = get_data(category='nutrition', report_name='Carbs', column_name='Carbohydrates (g)')

In [ ]:
fat = get_data(category='nutrition', report_name='Fat', column_name='Fat (g)')

In [ ]:
protein = get_data(category='nutrition', report_name='Protein', column_name='Protein (g)')

In [ ]:
exercise_min = get_data(category='fitness', report_name='Exercise Minutes', column_name='Exercise Minutes')

In [ ]:
calories_burned = get_data(category='fitness', report_name='Calories Burned', column_name='Exercise Calories')

In [ ]:
from functools import reduce
dfs = [calories, carbs, fat, protein, exercise_min, calories_burned]
data = reduce(lambda left, right: pd.merge(left, right, on='Date'), dfs)
data['Date'] = pd.to_datetime(data['Date'].astype(str) + '/2020')
data = data.set_index('Date')
data

In [ ]:
table1 = data[['Calories', 'Carbohydrates (g)', 'Fat (g)', 'Protein (g)']]
pd.concat(
    [
        consumedCaloriesDB[~consumedCaloriesDB.index.isin(table1.index)],
        table1
    ],
    axis=0,
    join='inner'
).to_sql('ConsumedCalories', con=engine, if_exists='replace')

In [ ]:
table2 = data[['Exercise Calories', 'Exercise Minutes']]
pd.concat(
    [
        exerciseSummaryDB[~exerciseSummaryDB.index.isin(table2.index)],
        table2
    ],
    axis=0,
    join='inner'
).to_sql('ExerciseSummary', con=engine, if_exists='replace')